In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json


from selenium import webdriver
from selenium.webdriver.common.by import By


In [4]:
# 카테고리 리스트
categories = ["Toners", "Moisturizers", "Sunscreen", "Serums%2C+Essence%2C+Ampoules", 
              "Acne+%26+Blemish+Treatments", "Exfoliating+Scrubs+%26+Peeling+Gel", "Eye+Cream+%26+Treatments"]

# 결과 저장용 리스트
all_icons = []

# 베이스 url
base_url = "https://www.skincarisma.com/search"

product_urls_per_category = {}

chrome = webdriver.Chrome()
time.sleep(5)

for category in categories:

    page_no = 1
    urls = []
    while True:

        url = f"{base_url}?category={category}&page={page_no}"
        # response = requests.get(url)
        chrome.get(url) 
        time.sleep(1)

        soup = BeautifulSoup(chrome.page_source, 'html.parser')

        li_list = soup.select('.list-unstyled.mt-2.mb-0 li')
        print(len(li_list))
        if len(li_list) == 0:
            break

        for li in li_list:
            # print( li.select('a')[0]['href'] )
            urls.append(li.select('a')[0]['href'])

        time.sleep(0.5)

        page_no += 1

    product_urls_per_category[category] = urls

chrome.quit()

25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
19
25
25
25
25
25
25
25
25
25
25
19
25
25
19
25
25
19
25
25
25
25
25
25
19
19
25
25
25
25
25
25
25
12
0
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
19
25
25
25
25
25
25
25
25
25
20
25
25
19
25
25
25
25
25
19
25
25
25
25
25
19
25
25
19
19
25
19
19
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25

In [9]:
print( len(product_urls_per_category) )
print( product_urls_per_category.keys())
for k, v in product_urls_per_category.items():
    print(len(v))
    print(v[0])

7
dict_keys(['Toners', 'Moisturizers', 'Sunscreen', 'Serums%2C+Essence%2C+Ampoules', 'Acne+%26+Blemish+Treatments', 'Exfoliating+Scrubs+%26+Peeling+Gel', 'Eye+Cream+%26+Treatments'])
2201
/products/purito/centella-green-level-calming-toner
7383
/products/aveeno/soothing-relief-moisture-cream-fragrance-free
2733
/products/estee-lauder/daywear-advanced-multi-protection-anti-oxidant-creme-broad-spectrum-spf-15
4767
/products/tia-m/vita-b3-source
1153
/products/stridex/single-step-acne-control-maximum-alcohol-free
1368
/products/skinfood/black-sugar-perfect-scrub-foam
1592
/products/clinique/smart-custom-repair-eye-treatment


In [ ]:
import pickle

with open('product_urls_per_category.pickle', 'wb') as f:
    pickle.dump(product_urls_per_category, f)

In [42]:
base_url2 = "https://www.skincarisma.com"

chrome = webdriver.Chrome()
time.sleep(5)



# 전체 { 제품명: 성분리스트 }를 저장하는 리스트 변수
product_data = []

for category, product_urls in product_urls_per_category.items():

    total = len(product_urls)

    for idx, url in enumerate(product_urls):
        full_url = f"{base_url2}{url}"
        
        # header로 크롤링 진행
        # headers = {
        #     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
        #     'Accept-Language': 'ko,en;q=0.9,en-US;q=0.8'
        # }
        # response = requests.get(url, headers=headers)
        # soup = BeautifulSoup(response.content.decode("utf-8"), 'html.parser')

        # chrome으로 크롤링 진행
        chrome.get(full_url)
        time.sleep(0.5)

        soup = BeautifulSoup(chrome.page_source, 'html.parser')
        
        # 제품명 읽기 
        product_name_tag = soup.find('h1', {'itemprop': 'name', 'class': 'card-title font-125'})
        product_name = product_name_tag.text.strip() if product_name_tag else "Unknown Product"

        # 성분 리스트 읽기
        span_tags = soup.select('.effect-wrapper .text-muted')
        ingredients = [{span.select_one('b').text.strip() : span.select_one('span').text.strip()} for span in span_tags if len(span.find_all('b')) > 0]

        # { 제품명 : 성분리스트 }를 리스트 변수에 저장
        product_data.append({
            "category": category,
            "product_name" : product_name,            
            "ingredients" : ingredients,
            "url": full_url
        })

        if (idx + 1) % 100 == 0:
            progress = (idx+1) / total * 100
            print(f"{category} : {idx + 1}/{total} 데이터 처리 완료 ({progress:.2f}%)")

with open('product_data.json', 'w', encoding='utf-8') as f:
    json.dump(product_data, f, ensure_ascii=False, indent=4)
        
chrome.quit()

Toners : 100/2201 데이터 처리 완료 (4.54%)
Toners : 200/2201 데이터 처리 완료 (9.09%)
Toners : 300/2201 데이터 처리 완료 (13.63%)
Toners : 400/2201 데이터 처리 완료 (18.17%)
Toners : 500/2201 데이터 처리 완료 (22.72%)
Toners : 600/2201 데이터 처리 완료 (27.26%)
Toners : 700/2201 데이터 처리 완료 (31.80%)
Toners : 800/2201 데이터 처리 완료 (36.35%)
Toners : 900/2201 데이터 처리 완료 (40.89%)
Toners : 1000/2201 데이터 처리 완료 (45.43%)
Toners : 1100/2201 데이터 처리 완료 (49.98%)
Toners : 1200/2201 데이터 처리 완료 (54.52%)
Toners : 1300/2201 데이터 처리 완료 (59.06%)
Toners : 1400/2201 데이터 처리 완료 (63.61%)
Toners : 1500/2201 데이터 처리 완료 (68.15%)
Toners : 1600/2201 데이터 처리 완료 (72.69%)
Toners : 1700/2201 데이터 처리 완료 (77.24%)
Toners : 1800/2201 데이터 처리 완료 (81.78%)
Toners : 1900/2201 데이터 처리 완료 (86.32%)
Toners : 2000/2201 데이터 처리 완료 (90.87%)
Toners : 2100/2201 데이터 처리 완료 (95.41%)
Toners : 2200/2201 데이터 처리 완료 (99.95%)
Moisturizers : 100/7383 데이터 처리 완료 (1.35%)
Moisturizers : 200/7383 데이터 처리 완료 (2.71%)
Moisturizers : 300/7383 데이터 처리 완료 (4.06%)
Moisturizers : 400/7383 데이터 처리 완료 (5.42%)
Moistur

In [43]:
import pickle

with open('product_urls_per_category.pickle', 'wb') as f:
    pickle.dump(product_urls_per_category, f)

with open('product_data.pickle', 'wb') as f:
    pickle.dump(product_data, f)